In [72]:
import numpy as np
import pandas as pd

import time
from datetime import datetime as dt

import matplotlib.pyplot as plt
import plotly
import plotly.graph_objs as go


# 1. Data Preparation

3 datasets:

S&P 500 daily adjusted price

Mean-variance weights table

Min-variance weights table

In [7]:
data_adj = pd.read_csv("1-sp500_adj_price.csv")
weight_meanv = pd.read_excel('mean_weighted_user8.xlsx')
weight_minv = pd.read_excel('minimum_weighted_user8.xlsx')
weight_equal=pd.read_excel('equally_weighted_user8.xlsx')

In [10]:
weight_meanv.head()

,tic,predicted_return,weights,trade_date
0,EOG,0.033723,0.000186,19950601
1,EQT,0.037745,0.000000,19950601
2,HES,0.051450,0.005125,19950601
3,NFX,0.030283,0.050000,19950601
4,OKE,0.041020,0.000000,19950601


In [11]:
weight_minv.head()

,tic,predicted_return,weights,trade_date
0,EOG,0.033723,0.009718,19950601
1,EQT,0.037745,0.000000,19950601
2,HES,0.051450,0.007371,19950601
3,NFX,0.030283,0.050000,19950601
4,OKE,0.041020,0.000000,19950601


In [12]:
weight_equal.head()

,tic,predicted_return,weights,trade_date
0,EOG,0.033723,0.006993,19950601
1,EQT,0.037745,0.006993,19950601
2,HES,0.051450,0.006993,19950601
3,NFX,0.030283,0.006993,19950601
4,OKE,0.041020,0.006993,19950601


In [18]:
#the length of the 3 tables should be the same
print(len(weight_meanv),
len(weight_minv),
len(weight_equal))


12932 12932 12932


## 1.1 Get stock name, trade date, daily date

Add 20170901 as tradedate_full


In [36]:
#get unique stock name from table
#stock names are the same between mean variance and minimum variance
#only the weights are different
stocks_name = list(set(weight_meanv['tic']))
stocks_name.sort()

In [37]:
len(stocks_name)

982

In [33]:
#get all daily date
#convert to list from pandas.core.series.Series
#use set to get unique values
daily = list(set(data_adj['datadate']))
#sort it
daily.sort()
print(type(data_adj['datadate']),type(daily))

<class 'pandas.core.series.Series'> <class 'list'>


In [31]:
len(daily)


7155

In [38]:
#get unique trade date from table
tradedate = list(set(weight_meanv['trade_date']))
tradedate.sort()


In [40]:
len(tradedate)

89

In [41]:
#add the last trade date 20170901
tradedate_full = tradedate.copy()
tradedate_full.append(20170901)
len(tradedate_full)

90

## 1.2 Get the daily price matrix based on stock_name

In [ ]:
#30mins running time
##################match current all stocks to daily adjusted price table#######################
# backtest daily price matrix:
# covert original table to matrix
# rows = all stocks: 971, columns = all dates:7155 days
#
start_time = time.time()

balance_daily = pd.DataFrame(index = stocks_name, columns = daily)
balance_daily = balance_daily.fillna(np.nan)

for i in stocks_name:
    ind = np.where(data_adj['tic'] == i)[0]
    tmp = data_adj.loc[ind, :]
    tmp.index = range(tmp.shape[0])
    for j in daily:
        ind2 = np.where(tmp['datadate'] == j)[0]
        if(len(ind2) != 0):
            balance_daily.loc[i,j] = tmp.loc[ind2[0], 'adj_price']

elapsed_time = time.time() - start_time
print(elapsed_time)
#writer2 = pd.ExcelWriter('balance_daily.xlsx')
#writer2 = pd.ExcelWriter('balance_daily_user8.xlsx')

#balance_daily.to_excel(writer2,'Sheet1')
#writer2.save()
###############################################

In [ ]:
balance_daily = pd.read_excel('balance_daily_user8.xlsx')


# 2. Function: calculate the rebalance matrix:

1. shares table
2. cost table
3. cash table
4. portfolio table
5. portfolio_cost table
6. portfolio_return table
7. portfolio_cumsum table

In [111]:
# set initial capital to 1 million and transaction percet to 0.1%

capital = 1000000
transaction_percent = 1/1000

In [232]:
#construct an empty matrix
#rows are stocks: 971 unique stocks
#columns are trade dates
#name the index as stock names

def cal_portfolio(stocks_name,tradedate, weight_table, capital, transaction_percent):


    balance_weight = pd.DataFrame(index = stocks_name, columns = tradedate)
    balance_weight = balance_weight.fillna(0)
    
    #loop through the weights table to fill the matrix
    #need weight tabele
    #0. weights table
    for i in range(weight_table.shape[0]):
        tics = weight_table.loc[i, 'tic']
        date = weight_table.loc[i, 'trade_date']
        w = weight_table.loc[i, 'weights']
        balance_weight.loc[tics, date] = w
        
    #1. shares table
    balance_share = pd.DataFrame(index = stocks_name, columns = tradedate)
    balance_share['initial'] = 0
    balance_share = balance_share.fillna(0)
    #2. costs table
    balance_cost = pd.DataFrame(index = stocks_name, columns = tradedate)
    balance_cost = balance_cost.fillna(0)
    #3. cash table
    balance_cash = pd.DataFrame(index = stocks_name, columns = tradedate)
    balance_cash = balance_cash.fillna(0)

    #4. portfolio without transaction costs table 
    portfolio = pd.DataFrame(index = range(1), columns = tradedate_full)
    portfolio.loc[0,tradedate[0]] = capital
    
    
    # calculate portfolio value without cost
    balance_price_full = []
    balance_price_full = balance_daily[tradedate_full]
    balance_price = balance_daily[tradedate]
    for j in range(len(tradedate)):
        i = tradedate[j]
        #current capital
        cap = portfolio.loc[0, i]
        #
        balance_cash[i] = cap * balance_weight[i]
        balance_share[i] = list(np.array(balance_cash[i])
                                      / np.array(balance_price[i]))
        balance_share[i] = balance_share[i].fillna(0)
        balance_share[i] = balance_share[i].astype(int)
        #next quarter price * current share to calculate next quarter total account value
        a = balance_price_full[tradedate_full[j+1]].fillna(0)
        portfolio.loc[0, tradedate_full[j+1]] = np.dot(balance_share[i], a)

    balance_cost = abs(np.diff(balance_share, axis = 1))
    balance_cost = balance_cost * balance_price
    balance_cost = np.sum(balance_cost, axis = 0) * transaction_percent
    balance_cost['20170901'] = 0
    
    #5. portfolio with transaction cost table
    #calcualte portolio value with transaction cost
    balance_cost.index = tradedate_full
    portfolio_cost = []
    portfolio_cost = portfolio.loc[0,] - balance_cost
    portfolio_cost.index = tradedate_full
    portfolio_cost_value = portfolio_cost.copy()
    
    #6. portfolio return P&L table
    portfolio_cumsum = portfolio_cost_value.copy()
    portfolio_cumsum.index = range(len(portfolio_cumsum))
    tmp = portfolio_cumsum.shift(1)
    tmp[0] = 0
    tmp.index = tradedate_full

    portfolio_cumsum = portfolio_cost_value- tmp
    portfolio_cumsum.index = range(len(portfolio_cumsum))
    tmp.index = range(len(tmp))
    for i in range(len(portfolio_cumsum)):
        if(tmp[i] != 0):
            portfolio_cumsum[i] = portfolio_cumsum[i]/tmp[i]

    portfolio_cumsum.index = range(len(portfolio_cumsum))
    portfolio_cumsum[0] = 0
    portfolio_return=portfolio_cumsum.copy()



    # 7. portfolio accumulative return table
    initial = 0
    for i in range(portfolio_cumsum.shape[0]):
        portfolio_cumsum[i] = initial + portfolio_cumsum[i]
        initial = portfolio_cumsum[i]
    
    #1. shares table
    #2. costs table
    #3. cash table
    #4. portfolio without transaction costs table 
    #5. portfolio with transaction cost table
    #6. portfolio return P&L table
    #7. portfolio accumulative return table

    return(balance_share,
           balance_cost,
           balance_cash,
           portfolio,
           portfolio_cost,
           portfolio_return,
           portfolio_cumsum)


In [224]:
#calculate mean variance allocation portolio
(balance_share_meanv,
 balance_cost_meanv,
 balance_cash_meanv,
 portfolio_meanv,
 portfolio_cost_meanv,
 portfolio_return_meanv,
 portfolio_cumsum_meanv)=cal_portfolio(stocks_name,tradedate, weight_meanv, capital, transaction_percent)


In [225]:
sum(portfolio_return_meanv)

3.7725457672472325

In [226]:
#calculate minimum variance allocation portolio
(balance_share_minv,
 balance_cost_minv,
 balance_cash_minv,
 portfolio_minv,
 portfolio_cost_minv,
 portfolio_return_minv,
 portfolio_cumsum_minv)=cal_portfolio(stocks_name,tradedate, weight_minv, capital, transaction_percent)

In [230]:
#calculate equally weighted allocation portolio
(balance_share_equal,
 balance_cost_equal,
 balance_cash_equal,
 portfolio_equal,
 portfolio_cost_equal,
 portfolio_return_equal,
 portfolio_cumsum_equal)=cal_portfolio(stocks_name,tradedate, weight_equal, capital, transaction_percent)

In [231]:
sum(portfolio_return_equal)

3.9190887589936798

# 3. Calculate SPX Portfolio Value, Return and Cumulative Return

In [256]:
# spx
spx = pd.read_excel("1-spx_price.xlsx")
spx_daily = list(set(spx['datadate']))
spx_daily.sort()
spx = spx['AdjClose']
spx.index = spx_daily

spx = spx[tradedate_full]
spx.index = range(len(spx))
spx_return = (spx - spx.shift(1))/spx
spx_return[0] = 0

spx_return.index = tradedate_full
spx_return.index = range(len(spx))
spx_cumsum=spx_return.copy()

spx_value = spx_return.copy()
initial = 0
for i in range(len(spx_cumsum)):
    spx_cumsum[i] = initial + spx_cumsum[i]
    initial = spx_cumsum[i]

initial = capital
for i in range(len(spx_value)):
    spx_value[i] = initial * (1 + spx_value[i])
    initial = spx_value[i]

spx_cumsum.index = tradedate_full
spx_value.index = tradedate_full

In [260]:
sum(spx_return)

1.167518238034603

# 4. Plot the Portfolio Performance

In [66]:
#convert tradedate_full to real date 
time_ind = pd.Series(tradedate_full)
for i in range(len(time_ind)):
    b = dt.strptime(str(time_ind[i]), '%Y%m%d')
    time_ind[i] = b.strftime('%Y-%m-%d')

In [287]:
time_ind.head()

0    1995-06-01
1    1995-09-01
2    1995-12-01
3    1996-03-01
4    1996-06-03
dtype: object

## 4.1 Plot Portfolio Value with Transaction Cost

In [261]:
trace0 = go.Scatter(x = time_ind, y = portfolio_cost_meanv, mode = 'lines', name = 'Mean-Variance')
trace1 = go.Scatter(x = time_ind, y = portfolio_cost_equal, mode = 'lines', name = 'Equally Weighted')
trace2 = go.Scatter(x = time_ind, y = portfolio_cost_minv, mode = 'lines', name = 'Min-Variance')
trace3 = go.Scatter(x = time_ind, y = spx_value, mode = 'lines', name = 'SPX')

data = [trace0, trace1, trace2,trace3]
layout = dict(title = "Portfolio Value Performance Quarterly with Transaction cost", 
              xaxis = dict(title = 'Quarterly Trade Date'), 
              yaxis = dict(title = 'Portfolio Value'))
fig = dict(data = data, layout = layout)

plotly.offline.iplot(fig)

#portfolio_tmp = portfolio_cost_value

## 4.2 Plot Portfolio Return

In [289]:
trace_return1 = go.Scatter(x = time_ind, y = portfolio_return_meanv, mode = 'lines', name = 'Mean-Variance')
trace_return2 = go.Scatter(x = time_ind, y = portfolio_return_minv, mode = 'lines', name = 'Minimum-Variance')
trace_return3 = go.Scatter(x = time_ind, y = portfolio_return_equal, mode = 'lines', name = 'Equally Weighted')
trace_return4 = go.Scatter(x = time_ind, y = spx_return, mode = 'lines', name = 'SPX')

data = [trace_return1,trace_return2,trace_return3,trace_return4]
layout = dict(title = "Portfolio Return Quarterly", 
              xaxis = dict(title = 'Quarterly Trade Date'), 
              yaxis = dict(title = 'Portfolio Value'))
fig = dict(data = data, layout = layout)
plotly.offline.iplot(fig)

## 4.3 Plot Portfolio Cumulative Return

In [285]:
trace_return1 = go.Scatter(x = time_ind, y = portfolio_cumsum_meanv, mode = 'lines', name = 'Mean-Variance')
trace_return2 = go.Scatter(x = time_ind, y = portfolio_cumsum_minv, mode = 'lines', name = 'Minimum-Variance')
trace_return3 = go.Scatter(x = time_ind, y = portfolio_cumsum_equal, mode = 'lines', name = 'Equally Weighted')
trace_return4 = go.Scatter(x = time_ind, y = spx_cumsum, mode = 'lines', name = 'SPX')

data = [trace_return1,trace_return2,trace_return3,trace_return4]
layout = dict(title = "Portfolio Cumulative Return Quarterly", 
              xaxis = dict(title = 'Quarterly Trade Date'), 
              yaxis = dict(title = 'Portfolio Value'))
fig = dict(data = data, layout = layout)
plotly.offline.iplot(fig)

# 5. Get all Quarterly Return and Calcualte Annulized return, standard deviation and Sharpe ratio

In [280]:
quarter_return = pd.DataFrame(index = range(len(spx_return)), columns = ['Mean-Var', 'Equally', 'Min-Var', 'SPX'])
quarter_return['SPX'] = spx_return
quarter_return['Mean-Var'] = portfolio_return_meanv
quarter_return['Equally'] = portfolio_return_equal
quarter_return['Min-Var'] = portfolio_return_minv

#writer_return = pd.ExcelWriter('quarter_return_user8.xlsx')
quarter_return.index=tradedate_full
#quarter_return.to_excel(writer_return,'Sheet1')
#writer_return.save()

In [348]:
quarter_return.head()

,Mean-Var,Equally,Min-Var,SPX
19950601,0,0,0,0.000000
19950901,0.124995,0.108626,0.104287,0.053827
19951201,0.0491306,0.0465689,0.0733228,0.071073
19960301,0.087531,0.0534956,0.0854701,0.058026
19960603,0.123325,0.111377,0.100025,0.034912


In [349]:
annulizedR_spx=quarter_return['SPX'].mean()*4
annulizedR_meanv=quarter_return['Mean-Var'].mean()*4
annulizedR_equal=quarter_return['Equally'].mean()*4
annulizedR_minv=quarter_return['Min-Var'].mean()*4
annulizedS_spx=quarter_return['SPX'].std()*2
annulizedS_meanv=quarter_return['Mean-Var'].std()*2
annulizedS_equal=quarter_return['Equally'].std()*2
annulizedS_minv=quarter_return['Min-Var'].std()*2

In [350]:
print("SPX",annulizedR_spx,annulizedS_spx)
print("Mean-Var",annulizedR_meanv,annulizedS_meanv)
print("Equally",annulizedR_equal,annulizedS_equal)
print("Min-Var",annulizedR_minv,annulizedS_minv)


SPX 0.05188969946820458 0.18846361580122992
Mean-Var 0.167668700767 0.230322449769
Equally 0.174181722622 0.253878788234
Min-Var 0.144642843088 0.164655054005


In [351]:
risk_free=0.015

In [352]:
sharpe_spx=(annulizedR_spx-risk_free)/annulizedS_spx
sharpe_meanv=(annulizedR_meanv-risk_free)/annulizedS_meanv
sharpe_equal=(annulizedR_equal-risk_free)/annulizedS_equal
sharpe_minv=(annulizedR_minv-risk_free)/annulizedS_minv

In [353]:
print("SPX Sharpe",sharpe_spx)
print("Mean-Var Sharpe",sharpe_meanv)
print("Equally Sharpe",sharpe_equal)
print("Min-Var Sharpe",sharpe_minv)


SPX Sharpe 0.19573910492681865
Mean-Var Sharpe 0.662847676898
Equally Sharpe 0.626998906562
Min-Var Sharpe 0.787360241511


In [341]:
insample_date=tradedate[1:51]

In [355]:
insample_date

[19950901,
 19951201,
 19960301,
 19960603,
 19960903,
 19961202,
 19970303,
 19970602,
 19970902,
 19971201,
 19980302,
 19980601,
 19980901,
 19981201,
 19990301,
 19990601,
 19990901,
 19991201,
 20000301,
 20000601,
 20000901,
 20001201,
 20010301,
 20010601,
 20010904,
 20011203,
 20020301,
 20020603,
 20020903,
 20021202,
 20030303,
 20030602,
 20030902,
 20031201,
 20040301,
 20040601,
 20040901,
 20041201,
 20050301,
 20050601,
 20050901,
 20051201,
 20060301,
 20060601,
 20060901,
 20061201,
 20070301,
 20070601,
 20070904,
 20071203]

In [356]:
quarter_return['SPX'][insample_date]


19950901    0.053827
19951201    0.071073
19960301    0.058026
19960603    0.034912
19960903   -0.019795
19961202    0.134609
19970303    0.048723
19970602    0.060317
19970902    0.087561
19971201    0.048411
19980302    0.069610
19980601    0.039671
19980901   -0.097278
19981201    0.154023
19990301    0.049249
19990601    0.044890
19990901    0.027654
19991201    0.047685
20000301   -0.013435
20000601    0.048053
20000901    0.047318
20001201   -0.156277
20010301   -0.059618
20010601    0.015420
20010904   -0.112742
20011203   -0.002690
20020301    0.001661
20020603   -0.087539
20020903   -0.185258
20021202    0.060469
20030303   -0.119452
20030602    0.136701
20030902    0.053807
20031201    0.044976
20040301    0.074267
20040601   -0.031011
20040901   -0.013826
20041201    0.071733
20050301    0.015730
20050601   -0.006812
20050901    0.015856
20051201    0.034064
20060301    0.020577
20060601   -0.004301
20060901    0.019298
20061201    0.061358
20070301    0.004604
20070601    0

In [347]:
annulizedR_spx_insample=quarter_return['SPX'][insample_date].mean()*4
annulizedR_meanv_insample=quarter_return['Mean-Var'][insample_date].mean()*4
annulizedR_equal_insample=quarter_return['Equally'][insample_date].mean()*4
annulizedR_minv_insample=quarter_return['Min-Var'][insample_date].mean()*4
annulizedS_spx_insample=quarter_return['SPX'][insample_date].std()*2
annulizedS_meanv_insample=quarter_return['Mean-Var'][insample_date].std()*2
annulizedS_equal_insample=quarter_return['Equally'][insample_date].std()*2
annulizedS_minv_insample=quarter_return['Min-Var'][insample_date].std()*2
print("SPX",annulizedR_spx_insample,annulizedS_spx_insample)
print("Mean-Var",annulizedR_meanv_insample,annulizedS_meanv_insample)
print("Equally",annulizedR_equal_insample,annulizedS_equal_insample)
print("Min-Var",annulizedR_minv_insample,annulizedS_minv_insample)
sharpe_spx_insample=(annulizedR_spx_insample-risk_free)/annulizedS_spx_insample
sharpe_meanv_insample=(annulizedR_meanv_insample-risk_free)/annulizedS_meanv_insample
sharpe_equal_insample=(annulizedR_equal_insample-risk_free)/annulizedS_equal_insample
sharpe_minv_insample=(annulizedR_minv_insample-risk_free)/annulizedS_minv_insample
print("SPX Sharpe",sharpe_spx_insample)
print("Mean-Var Sharpe",sharpe_meanv_insample)
print("Equally Sharpe",sharpe_equal_insample)
print("Min-Var Sharpe",sharpe_minv_insample)

SPX 0.0711785925475649 0.13828636298362326
Mean-Var 0.195272903073 0.157365362189
Equally 0.199931318893 0.166430681877
Min-Var 0.174051597452 0.139022544778
SPX Sharpe 0.4062482470105741
Mean-Var Sharpe 1.14556914282
Equally Sharpe 1.11116121624
Min-Var Sharpe 1.14407053694
